In [ ]:
 from google.colab import drive
 drive.mount('/content/drive')
 import sys
 sys.path.append('/content/drive/My Drive/Colab Notebooks')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pathlib
pathlib.Path().resolve()

PosixPath('/content')

In [ ]:
sample = '/content/drive/My Drive/Colab Notebooks/umkm'

In [ ]:
pip install tensorflow

In [ ]:
pip install keras

In [ ]:
pip install transformers

In [ ]:
pip install python-telegram-bot==13.4.1

In [ ]:
import torch
import tensorflow
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

In [ ]:
# Load model dan tokenizer
model_name = "Rifky/IndoBERT-Large-P2-QA"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name, from_tf=True)


# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("Rifky/IndoBERT-Large-P2-QA")

# Contoh teks pertanyaan
question = "Apa itu UMKM?"

# Contoh teks konteks\
context = "UMKM (Usaha Mikro, Kecil, dan Menengah) adalah suatu bentuk usaha yang memiliki ciri-ciri berikut..."

# Tokenisasi pertanyaan
question_tokens = tokenizer.tokenize(question)
print("Tokenisasi pertanyaan:", question_tokens)

# Tokenisasi konteks
context_tokens = tokenizer.tokenize(context)
print("Tokenisasi konteks:", context_tokens)

# Encode pertanyaan dan konteks menjadi indeks token
encoded_inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")
input_ids = encoded_inputs["input_ids"]
attention_mask = encoded_inputs["attention_mask"]
print("Indeks token:", input_ids)
print("Masking attention:", attention_mask)

All TF 2.0 model weights were used when initializing BertForQuestionAnswering.

All the weights of BertForQuestionAnswering were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForQuestionAnswering for predictions without further training.


Tokenisasi pertanyaan: ['apa', 'itu', 'umkm', '?']
Tokenisasi konteks: ['umkm', '(', 'usaha', 'mikro', ',', 'kecil', ',', 'dan', 'menengah', ')', 'adalah', 'suatu', 'bentuk', 'usaha', 'yang', 'memiliki', 'ciri', '-', 'ciri', 'berikut', '.', '.', '.']
Indeks token: tensor([[    2,   387,   137, 13839, 30477,     3, 13839, 30464,  1062,  6782,
         30468,   855, 30468,    41,  3882, 30465,   154,   607,   902,  1062,
            34,   343,  2878, 30469,  2878,   713, 30470, 30470, 30470,     3]])
Masking attention: tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1]])


In [ ]:
# Load dataset dalam format txt
dataset_path = "/content/drive/My Drive/Colab Notebooks/umkm.txt"
try:
    with open(dataset_path, "r", encoding="utf-8") as file:
        dataset = file.read()
except FileNotFoundError:
    print("File not found. Please check the file path.")
    exit()

# Menampilkan isi dataset
print("Isi dataset:")
print(dataset)


Isi dataset:
UMKM adalah usaha yang meliputi usaha mikro, kecil, dan menengah. Usaha mikro adalah usaha dengan jumlah karyawan kurang dari 10 orang dan aset tidak lebih dari Rp50 juta. Usaha kecil memiliki jumlah karyawan antara 10 hingga 50 orang dan aset antara Rp50 juta hingga Rp500 juta. Sementara itu, usaha menengah memiliki jumlah karyawan antara 51 hingga 500 orang dan aset antara Rp500 juta hingga Rp10 miliar. UMKM mencakup berbagai sektor seperti pertanian, perdagangan, jasa, industri, dan lain-lain.Usaha Mikro, Kecil, dan Menengah (UMKM) memainkan peranan yang sangat penting dalam perekonomian global. Mereka adalah tulang punggung ekonomi yang berkontribusi pada pertumbuhan ekonomi, penciptaan lapangan kerja, inovasi, dan pengurangan kesenjangan sosial. Dalam esai ini, kita akan mengeksplorasi peran UMKM dalam perekonomian dan pentingnya mereka dalam mencapai pembangunan berkelanjutan.UMKM merupakan tulang punggung ekonomi yang berperan dalam perekonomian dan pembangunan berk

new training

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from collections import Counter

# Load the pretrained model and tokenizer
model_name = "Rifky/IndoBERT-Large-P2-QA"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name, from_tf=True)

# Evaluation dataset
evaluation_data = [
    {
        "context": "UMKM merupakan sektor penting dalam perekonomian Indonesia. Usaha mikro, kecil, dan menengah banyak berperan dalam menciptakan lapangan kerja dan pertumbuhan ekonomi. Salah satu contoh UMKM yang sukses di Indonesia adalah usaha kerajinan tangan yang terkenal dari daerah Jepara. Produk kerajinan tangan seperti ukiran kayu dan mebel dari Jepara telah dikenal baik di dalam maupun luar negeri.",
        "question": "Apa contoh produk kerajinan tangan dari Jepara?",
        "ground_truth": "ukiran kayu dan mebel"
    },
    {
        "context": "Pada saat pandemi COVID-19, UMKM menghadapi banyak tantangan dalam menjalankan usahanya. Salah satu kendala yang dihadapi adalah sulitnya mendapatkan modal usaha. Pemerintah Indonesia telah memberikan berbagai dukungan untuk membantu UMKM, seperti program kredit usaha rakyat (KUR) dan bantuan modal usaha.",
        "question": "Apa program yang disediakan pemerintah untuk membantu modal usaha UMKM?",
        "ground_truth": "kredit usaha rakyat (KUR) dan bantuan modal usaha"
    },
    {
        "context": "Pemasaran merupakan hal penting dalam mengembangkan usaha UMKM. Dalam era digital, banyak UMKM memanfaatkan media sosial sebagai sarana pemasaran. Dengan menggunakan platform seperti Instagram dan Facebook, UMKM dapat mempromosikan produknya kepada calon konsumen dengan lebih mudah dan efisien.",
        "question": "Platform apa yang sering digunakan UMKM untuk pemasaran produk?",
        "ground_truth": "Instagram dan Facebook"
    },
    {
        "context": "UMKM di Indonesia dapat memperoleh dukungan dari berbagai lembaga dan program pemerintah. Salah satu contohnya adalah Bank Indonesia yang memiliki program 'UMKM Go Online' untuk membantu UMKM dalam memasarkan produk secara online dan meningkatkan aksesibilitas konsumen.",
        "question": "Apa program Bank Indonesia untuk mendukung UMKM dalam memasarkan produk secara online?",
        "ground_truth": "'UMKM Go Online'"
    },
    {
        "context": "Pelatihan dan pendidikan merupakan faktor penting dalam pengembangan UMKM. Dengan pengetahuan dan keterampilan yang cukup, UMKM dapat meningkatkan kualitas produk dan inovasi bisnis. Beberapa lembaga seperti Kementerian Koperasi dan UKM menyelenggarakan program pelatihan dan pendidikan khusus untuk UMKM.",
        "question": "Lembaga apa yang menyelenggarakan program pelatihan dan pendidikan khusus untuk UMKM?",
        "ground_truth": "Kementerian Koperasi dan UKM"
    },
    {
        "context": "Keberlanjutan adalah aspek penting yang perlu diperhatikan dalam pengembangan UMKM. UMKM yang berfokus pada praktik bisnis yang berkelanjutan dan ramah lingkungan dapat mendapatkan manfaat jangka panjang. Misalnya, UMKM yang menggunakan bahan baku daur ulang atau praktik produksi yang hemat energi.",
        "question": "Apa manfaat jangka panjang bagi UMKM yang berfokus pada praktik bisnis yang berkelanjutan dan ramah lingkungan?",
        "ground_truth": "manfaat jangka panjang"
    }
]

f1_scores = []
exact_match = 0

for example in evaluation_data:
    context = example["context"]
    question = example["question"]
    ground_truth = example["ground_truth"]

    # Tokenize the context and question
    inputs = tokenizer.encode_plus(question, context, return_tensors="pt", max_length=512, truncation=True)

    # Get the model's predicted answer
    with torch.no_grad():
        outputs = model(**inputs)
        start_scores = outputs.start_logits
        end_scores = outputs.end_logits

    # Find the start and end indices of the predicted answer
    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores)

    # Convert token indices to tokens
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze().tolist())
    answer = ' '.join(tokens[start_index:end_index + 1]).replace("Ġ", "").strip()

    # Calculate F1 score
    common = Counter(answer.lower().split()) & Counter(ground_truth.lower().split())
    num_common = sum(common.values())
    if num_common == 0:
        f1_score = 0
    else:
        precision = num_common / len(answer.split())
        recall = num_common / len(ground_truth.split())
        f1_score = 2 * (precision * recall) / (precision + recall)
    f1_scores.append(f1_score)

    # Check if the predicted answer is an exact match
    if answer.lower() == ground_truth.lower():
        exact_match += 1

# Calculate average F1 score
avg_f1_score = sum(f1_scores) / len(f1_scores)

# Calculate EM percentage
em_percentage = (exact_match / len(evaluation_data)) * 100

print("Average F1 Score:", avg_f1_score)
print("Exact Match Percentage:", em_percentage)


All TF 2.0 model weights were used when initializing BertForQuestionAnswering.

All the weights of BertForQuestionAnswering were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForQuestionAnswering for predictions without further training.


Average F1 Score: 0.8439153439153438
Exact Match Percentage: 66.66666666666666


In [ ]:
# Fungsi untuk membagi teks menjadi bagian-bagian yang lebih kecil
def split_text(teks, max_length):
    bagian_teks = []
    awal = 0
    while awal < len(teks):
        bagian = teks[awal: awal + max_length]
        bagian_teks.append(bagian)
        awal += max_length
    return bagian_teks


In [ ]:
# Fungsi untuk menjawab pertanyaan
def answer_question(question, context):
    # Pra-pemrosesan pertanyaan
    encoded_inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")
    input_ids = encoded_inputs["input_ids"]
    attention_mask = encoded_inputs["attention_mask"]

    # Melakukan prediksi jawaban
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    # Mengambil indeks awal dan akhir jawaban yang memiliki skor tertinggi
    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores)

    # Mengkonversi indeks token menjadi teks jawaban
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    answer = tokenizer.convert_tokens_to_string(tokens[start_index : end_index + 1])

    return answer

 # Test the model
question = "Apa itu UMKM?"
context = "UMKM (Usaha Mikro, Kecil, dan Menengah) adalah suatu bentuk usaha yang memiliki ciri-ciri berikut..."
answer = answer_question(question, context)
print("Answer:", answer)

Answer: usaha mikro, kecil, dan menengah )


In [ ]:
# Fungsi untuk menghandle pertanyaan dari pengguna
def handle_question(update: Update, context: CallbackContext):
    question = update.message.text

    # Memanggil fungsi answer_question untuk mendapatkan jawaban
    answer_parts = []
    text_parts = split_text(dataset, max_length=512)
    for part in text_parts:
        answer_part = answer_question(question, part)
        answer_parts.append(answer_part)

    # Menggabungkan jawaban dari bagian-bagian teks
    answer = " ".join(answer_parts)

    # Mengirim jawaban ke pengguna
    if answer.strip():
        context.bot.send_message(chat_id=update.effective_chat.id, text=answer)
    else:
        context.bot.send_message(chat_id=update.effective_chat.id, text="Maaf, Mas UMKM tidak dapat menemukan jawaban,coba pertanyaan lain.")

In [ ]:
# Fungsi untuk menjalankan chatbot
def main():
    # Inisialisasi updater
    updater = Updater(token="6220994340:AAGQYsBWe6TI01aeZHyq0OL7Dm2deIqSTVw", use_context=True)

    # Menambahkan handler untuk command start
    def start(update: Update, context: CallbackContext):
        context.bot.send_message(chat_id=update.effective_chat.id, text="Halo Mas UMKM disini, silahkan ajukan pertanyaan anda.")
    updater.dispatcher.add_handler(CommandHandler("start", start))

    # Menambahkan handler untuk menerima pertanyaan dari pengguna
    updater.dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, handle_question))

    # Menjalankan bot
    updater.start_polling()
    updater.idle()

if __name__ == "__main__":
    main()

